# BERT - CRISPRoff

### 1. Packages and Data

In [263]:
import os
import json

n_sequences = 500
kmer_length = 5

#empty commit

In [264]:
import os
from subprocess import PIPE, run
import time
import pandas as pd
import numpy as np 
import pprint
import requests

pp = pprint.PrettyPrinter()

Data = pd.read_csv('/Users/sergiomares/Desktop/Nunez/Jin file/TSS_CpG_crispriphenotype_table.txt', delimiter = '\t',header = 0)

b = []

for i in range (n_sequences):
    datas = requests.get('http://togows.org/api/ucsc/hg19/'+ str(Data['chromosome'][i])+':'+str(int(Data["Primary TSS, 5'"][i]-1000))+'-'+str(Data["Primary TSS, 5'"][i]+1000)).text.replace('\n','')    
    b.append(datas)

Promoter_sequences = pd.DataFrame(b)

def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

Promoter_sequences = Promoter_sequences.apply(lambda x: getKmers(x[0], kmer_length), axis = 1)
Promoter_sequences = pd.DataFrame(Promoter_sequences)

Promoters_text = list(Promoter_sequences[0])

for item in range(len(Promoter_sequences)):
    Promoters_text[item] = ' '.join(Promoter_sequences[0][item])

y_data = Promoter_sequences.iloc[:, 0].values     

Promoter_sequences[1] = Data.index[:(len(Promoter_sequences))]
Promoter_sequences.columns = [['sequence', 'index']]
Promoter_sequences['text'] = pd.DataFrame(Promoters_text)

#dataset = dict(pd.MultiIndex.from_frame(Promoter_sequences[['index','text']]))

In [265]:
datasets = Dataset.from_pandas(Promoter_sequences[['index','text']])
datasets

Dataset({
    features: ["('index',)", "('text',)"],
    num_rows: 500
})

### 2. Building a tokenizer

In [266]:
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset:
      print(t, file=f)

dataset_to_text(datasets, "train.txt")

In [267]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files='train.txt', 
                vocab_size=30_522,
                min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>']) 

In [269]:
import os 

os.mkdir('./liberto')
tokenizer.save_model('liberto')

['liberto/vocab.json', 'liberto/merges.txt']

### 3. Initializing the tokenizer

In [270]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('liberto',max_length = 3)

Didn't find file liberto/added_tokens.json. We won't load it.
Didn't find file liberto/special_tokens_map.json. We won't load it.
Didn't find file liberto/tokenizer_config.json. We won't load it.
loading file liberto/vocab.json
loading file liberto/merges.txt
loading file None
loading file None
loading file None
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [271]:
tokens = tokenizer('tata')
tokens.input_ids

[0, 88, 370, 2]

### 4. Preparing the data

In [309]:
batch = tokenizer( Promoters_text , max_length=10, padding='max_length', truncation=True)

In [318]:
import torch

labels = torch.tensor([x for x in batch['input_ids']])
mask = torch.tensor([x for x in batch['attention_mask']])

In [311]:
labels

tensor([[   0, 1510,  262,  ..., 1184, 1395,    2],
        [   0,  409,  315,  ...,  543,  416,    2],
        [   0,  264,  357,  ..., 1256, 1238,    2],
        ...,
        [   0,  348,  283,  ...,  761,  623,    2],
        [   0, 1609,  970,  ...,  528,  764,    2],
        [   0, 1505,  283,  ...,  636,  817,    2]])

In [313]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
# mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
mask_arr = (rand < .15) * (input_ids > 2) 
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3 

In [314]:
input_ids.shape

torch.Size([500, 10])

In [319]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels} 

In [320]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [321]:
dataset = Dataset(encodings) 

In [322]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) 

### 5. Training the model

#### A. Initializing the model

In [323]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
    ) 

In [324]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config) 

#### B. Training Preparation

In [327]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device) 

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [328]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/Users/sergiomares/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### C. Training 

In [329]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item()) 

Epoch 1: 100%|██████████| 32/32 [00:38<00:00,  1.21s/it, loss=3.05]


In [330]:
model.save_pretrained('./liberto')  # and don't forget to save liBERTo!

Configuration saved in ./liberto/config.json
Model weights saved in ./liberto/pytorch_model.bin


### 6. The real test

In [331]:
from transformers import pipeline

In [332]:
fill = pipeline('fill-mask', model='liberto', tokenizer='liberto')

loading configuration file liberto/config.json
Model config RobertaConfig {
  "_name_or_path": "liberto",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file liberto/config.json
Model config RobertaConfig {
  "_name_or_path": "liberto",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token

In [335]:
fill(f'gcgc {fill.tokenizer.mask_token} tata ')

[{'score': 0.0014843828976154327,
  'token': 628,
  'token_str': ' acagg',
  'sequence': 'gcgc acagg tata '},
 {'score': 0.0008461986435577273,
  'token': 266,
  'token_str': 'ag',
  'sequence': 'gcgcag tata '},
 {'score': 0.0008192642126232386,
  'token': 1014,
  'token_str': ' cttgt',
  'sequence': 'gcgc cttgt tata '},
 {'score': 0.0007164615672081709,
  'token': 389,
  'token_str': ' ttttt',
  'sequence': 'gcgc ttttt tata '},
 {'score': 0.0006271416204981506,
  'token': 537,
  'token_str': ' tgctg',
  'sequence': 'gcgc tgctg tata '}]